In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from seaborn import heatmap
from sklearn import tree
import seaborn as sns
from keras.models import Sequential
#from preprocessing import preprocessing
import preprocessing as pp

In [3]:
#Aufruf py file

Y_df, X_df = pp.preprocessing()

In [29]:
#Train- data split into train and test datasets
x_train, x_test, y_train, y_test = train_test_split(X_df, Y_df, train_size=0.8, random_state=0, stratify=Y_df)

print("training samples: ", x_train.shape[0])
print("test samples: ", x_test.shape[0])
print("sample dimension: ", x_train.shape[1])

training samples:  700527
test samples:  175132
sample dimension:  26


In [5]:
X_df.columns



Index(['DayOfWeek', 'Weekend', 'Address', 'CoordinateClusters', 'year',
       'month', 'quarter', 'hour', 'minute', 'NORTHERN', 'PARK', 'INGLESIDE',
       'BAYVIEW', 'RICHMOND', 'CENTRAL', 'TARAVAL', 'TENDERLOIN', 'MISSION',
       'SOUTHERN', 'temp', 'Partially cloudy', 'Clear',
       'Rain, Partially cloudy', 'Overcast', 'Rain, Overcast', 'Rain'],
      dtype='object')

In [30]:
x_train = (x_train - x_train.mean())/ x_train.std() #immer Trainings Daten verwenden, auch bei den Testdaten

x_test = (x_test - x_train.mean())/ x_train.std()

In [31]:
from keras.models import Sequential
from keras.layers import Input, Flatten, Dense, Dropout

model = Sequential()

#220, 110, 0.2
model.add(Input(shape=X_df.shape[1]))
model.add(Dense(220, activation='relu'))
model.add(Dropout(rate=0.2))
model.add(Dense(110, activation='relu'))
model.add(Dropout(rate=0.2))
model.add(Dense(220, activation='relu'))
model.add(Dropout(rate=0.1))
model.add(Dense(39, activation='softmax'))


model.summary()


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 300)               8100      
                                                                 
 dropout_12 (Dropout)        (None, 300)               0         
                                                                 
 dense_17 (Dense)            (None, 150)               45150     
                                                                 
 dropout_13 (Dropout)        (None, 150)               0         
                                                                 
 dense_18 (Dense)            (None, 300)               45300     
                                                                 
 dropout_14 (Dropout)        (None, 300)               0         
                                                                 
 dense_19 (Dense)            (None, 39)               

In [32]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")

In [33]:
from keras.callbacks import EarlyStopping, LearningRateScheduler
early_stopping = EarlyStopping(
    monitor="val_loss",
    mode="min",
    patience=10,
)

In [34]:
import math

initial_learning_rate = 0.01
epochs = 100
decay = initial_learning_rate / epochs

def lr_time_based_decay(epoch, lr):
    return lr * 1 / (1 + decay * epoch)

history = model.fit(
  x_train,
  y_train,
  batch_size=200,
  epochs=100,
  verbose=1,
  validation_split=.2,
  callbacks=[early_stopping, LearningRateScheduler(lr_time_based_decay, verbose=1)]
)



Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/100
2803/2803 [==============================] - 19s 7ms/step - loss: 2.5216 - val_loss: 2.4624 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.0009999000574917021.
Epoch 2/100
2803/2803 [==============================] - 19s 7ms/step - loss: 2.4680 - val_loss: 2.4325 - lr: 9.9990e-04

Epoch 3: LearningRateScheduler setting learning rate to 0.000999700106714659.
Epoch 3/100
2803/2803 [==============================] - 18s 7ms/step - loss: 2.4470 - val_loss: 2.4132 - lr: 9.9970e-04

Epoch 4: LearningRateScheduler setting learning rate to 0.0009994003415259673.
Epoch 4/100
2803/2803 [==============================] - 19s 7ms/step - loss: 2.4305 - val_loss: 2.4030 - lr: 9.9940e-04

Epoch 5: LearningRateScheduler setting learning rate to 0.0009990007918579775.
Epoch 5/100
2803/2803 [==============================] - 20s 7ms/step - loss: 2.4218 - val_loss: 2.3959 - lr: 9.9900e-04


KeyboardInterrupt: 

In [ ]:
import keras
def plot_history(history: keras.callbacks.History):
  """
  plot the training and validation loss for each training epoch

  history: a History object, output of the .fit method of a keras model
  """
  n = len(history.history['loss'])
  plt.plot(np.arange(n), history.history['loss'], label="training loss")
  plt.plot(np.arange(n), history.history['val_loss'], label="validation loss")
  plt.xticks(range(0, n + 1, 2))
  plt.legend()
  plt.show()

In [ ]:
plot_history(history)

#### GradientBoostingClassifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier(n_estimators=150, max_depth=8, random_state=0).fit(x_train, y_train)


In [ ]:
y_preds = clf.predict_proba(x_test)
test_log_loss = log_loss(y_test, y_preds)
print("Testing Log Loss:", test_log_loss)

#### RandomForestClassifier

In [70]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=100, random_state=0, class_weight="balanced",n_estimators=150)
clf.fit(x_train, y_train)

#y_pred = clf.predict(x_test)


RandomForestClassifier(class_weight='balanced', max_depth=100, n_estimators=150,
                       random_state=0)

In [71]:
from sklearn.metrics import log_loss


y_preds = clf.predict_proba(x_test)
test_log_loss = log_loss(y_test, y_preds)
print("Testing Log Loss:", test_log_loss)


Testing Log Loss: 3.440130711844978
